In [2]:
from torch.nn import Module, ReLU, Conv2d, Linear, MaxPool2d, LogSoftmax, NLLLoss, Dropout, BatchNorm2d
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torch import flatten, float, no_grad
from torch.optim import Adam
import torch
import wandb
import math

In [3]:
PARAMETERS = {
    'data_augmentation': False,
    'batch_normalization': False,
    'filters': 32, # no. of filters in first layer
    'filter_org': 'same', # 'half', 'double'
    'dropout': 0,
    'activation': 'relu',
    'train_data_dir': "./data/train",
    'test_data_dir': "./data/val",
    'batch_size': 64,
    'learning_rate': 0.001,
    'epochs': 3,
    'dim': 256,
    'conv_kernel_size': 3
}

In [4]:
CLASSES = {
    0:'Amphibia',
    1:'Animalia',
    2:'Arachnida',
    3:'Aves',
    4:'Fungi',
    5:'Insecta',
    6:'Mammalia',
    7:'Mollusca',
    8:'Plantae',
    9:'Reptilia'
}

In [5]:
def get_data(param, type):
    if(type == 'train'):
        transform = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.3),
            transforms.RandomRotation(degrees=12),
            transforms.Resize((param['dim'],param['dim'])),
            transforms.ToTensor(), 
            transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])  
        ])

        tdataset = datasets.ImageFolder(root=param['train_data_dir'], transform=transform)
        total = len(tdataset)
        train_sample = math.ceil(total*(0.8))
        val_sample = total-train_sample
        # print(total, train_sample, val_sample)
        train_dataset, validation_dataset = torch.utils.data.random_split(tdataset, [train_sample, val_sample])
        train_dataloader = DataLoader(train_dataset, batch_size=param['batch_size'], shuffle=True)
        validation_dataloader = DataLoader(validation_dataset, batch_size=param['batch_size'], shuffle=False)
        return train_dataloader, validation_dataloader
    

In [6]:
# get_data(PARAMETERS, 'train')

In [7]:
class CNN(Module):
    def __init__(self, param):
        super(CNN, self).__init__()
        self.param=param
        self.data_augmentation = param['data_augmentation']
        self.batch_normalization = param['batch_normalization']
        self.dropout = param['dropout']
        self.activation = param['activation']
        self.filters = self.filter_logic(param['filters'], param['filter_org'])
        self.conv_ks = param['conv_kernel_size']
        self.dim = param['dim']
        self.dropout = param['dropout']

        curr_dim = self.dim
        self.conv1 = Conv2d(kernel_size=(self.conv_ks,self.conv_ks), in_channels=3, out_channels=self.filters[0])
        curr_dim -= (self.conv_ks-1)
        self.act1 = ReLU()
        self.pool1 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        curr_dim //= 2
        self.dropout1 = Dropout(p=self.dropout)

        self.conv2 = Conv2d(kernel_size=(self.conv_ks,self.conv_ks), in_channels=self.filters[0], out_channels=self.filters[1])
        curr_dim -= (self.conv_ks-1)
        self.act2 = ReLU()
        self.pool2 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        curr_dim //= 2
        self.dropout2 = Dropout(p=self.dropout)

        self.conv3 = Conv2d(kernel_size=(self.conv_ks,self.conv_ks), in_channels=self.filters[1], out_channels=self.filters[2])
        curr_dim -= (self.conv_ks-1)
        self.act3 = ReLU()
        self.pool3 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        curr_dim //= 2
        self.dropout3 = Dropout(p=self.dropout)

        self.conv4 = Conv2d(kernel_size=(self.conv_ks,self.conv_ks), in_channels=self.filters[2], out_channels=self.filters[3])
        curr_dim -= (self.conv_ks-1)
        self.act4 = ReLU()
        self.pool4 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        curr_dim //= 2
        self.dropout4 = Dropout(p=self.dropout)

        self.conv5 = Conv2d(kernel_size=(self.conv_ks,self.conv_ks), in_channels=self.filters[3], out_channels=self.filters[4])
        curr_dim -= (self.conv_ks-1)
        self.act5 = ReLU()
        self.pool5 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        curr_dim //= 2
        self.dropout5 = Dropout(p=self.dropout)

        self.fc1 = Linear(in_features=(curr_dim * curr_dim * self.filters[4]), out_features=(curr_dim * curr_dim * self.filters[4]))  # How to calculate dimension of filters at previous level
        self.act6 = ReLU()
        self.dropout6 = Dropout(p=0.5)
        
        self.out = Linear(in_features=(curr_dim * curr_dim * self.filters[4]), out_features=10)
        self.act7 = LogSoftmax(dim=1)

    
    def filter_logic(self, filter, org):
        level = []
        org = org.lower()
        if org == 'same':
            level = [filter for i in range(5)]
        elif org == 'double':
            level = [filter*pow(2,i) for i in range(5)]
        elif org == 'half':
            level = [max(filter//pow(2,i),1) for i in range(5)]
        return level

    

    def forward(self, r):

        r=self.conv1(r)
        r=self.act1(r)
        r=self.pool1(r)
        r=self.dropout1(r)

        r=self.conv2(r)
        r=self.act2(r)
        r=self.pool2(r)
        r=self.dropout2(r)

        r=self.conv3(r)
        r=self.act3(r)
        r=self.pool3(r)
        r=self.dropout3(r)

        r=self.conv4(r)
        r=self.act4(r)
        r=self.pool4(r)
        r=self.dropout4(r)

        r=self.conv5(r)
        r=self.act5(r)
        r=self.pool5(r)
        r=self.dropout5(r)

        r=flatten(r,1)
        r=self.fc1(r)
        r=self.act6(r)
        r=self.dropout6(r)
        
        r=self.out(r)
        output=self.act7(r)

        return output
        


In [8]:
# def train(param):
#     device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
#     model = CNN(param).to(device)
#     optimizer = Adam(model.parameters(), lr=param['learning_rate'])
#     loss_function = NLLLoss()
#     train_data_loader, validation_data_loader = get_data(param, 'train')
    

#     for _ in range(param['epochs']):
#         model.train()
#         totalTrainLoss = 0
#         totalValLoss = 0
#         trainCorrect = 0
#         valCorrect = 0
#         train_counter=0
#         validation_counter=0
#         for (image, label) in train_data_loader:
#             (image, label) = (image.to(device), label.to(device))
#             prediction = model(image)
#             loss = loss_function(prediction, label)
            
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
            
#             totalTrainLoss += loss
#             trainCorrect += (prediction.argmax(1) == label).type(float).sum().item()
#             train_counter+=1
#             print(train_counter)
        
#         with no_grad():
#             model.eval()
#             for (image, label) in validation_data_loader:
#                 (image, label) = (image.to(device), label.to(device))
#                 pred = model(image)
#                 totalValLoss += loss_function(pred, label)
#                 valCorrect += (pred.argmax(1) == label).type(float).sum().item()
#                 validation_counter+=1

#         print(f"Train Loss --> {(totalTrainLoss/train_counter).cpu().detach().numpy()}")
#         print(f"Train Accuracy --> {trainCorrect/len(train_data_loader.dataset)}")
#         print(f"Validation Loss --> {(totalValLoss/validation_counter).cpu().detach().numpy()}")
#         print(f"Validation Accuracy --> {valCorrect/len(validation_data_loader.dataset)}")

In [9]:
# train(PARAMETERS)

In [10]:
## for wandb sweeps
def train():
    wandb.init()
    param = wandb.config
    wandb.run.name = f'fltr_{param.filters}_fltrOrg_{param.filter_org}_dataAug_{param.data_augmentation}_batchNorm_{param.batch_normalization}_act_{param.activation}_batchSz_{param.batch_size}'

    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    model = CNN(param).to(device)
    optimizer = Adam(model.parameters(), lr=param['learning_rate'])
    loss_function = NLLLoss()
    train_data_loader, validation_data_loader = get_data(param, 'train')
    

    for epo in range(param['epochs']):
        model.train()
        totalTrainLoss = 0
        totalValLoss = 0
        trainCorrect = 0
        valCorrect = 0
        train_counter=0
        validation_counter=0
        for (image, label) in train_data_loader:
            (image, label) = (image.to(device), label.to(device))
            prediction = model(image)
            loss = loss_function(prediction, label)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            totalTrainLoss += loss
            trainCorrect += (prediction.argmax(1) == label).type(float).sum().item()
            train_counter+=1
            print(train_counter)
        
        with no_grad():
            model.eval()
            for (image, label) in validation_data_loader:
                (image, label) = (image.to(device), label.to(device))
                pred = model(image)
                totalValLoss += loss_function(pred, label)
                valCorrect += (pred.argmax(1) == label).type(float).sum().item()
                validation_counter+=1

        tr_ls = (totalTrainLoss/train_counter).cpu().detach().numpy()
        tr_acc = trainCorrect/len(train_data_loader.dataset)
        val_ls = (totalValLoss/validation_counter).cpu().detach().numpy()
        val_acc = valCorrect/len(validation_data_loader.dataset)
        print(f"Train Loss --> {tr_ls}")
        print(f"Train Accuracy --> {tr_acc}")
        print(f"Validation Loss --> {val_ls}")
        print(f"Validation Accuracy --> {val_acc}")
        
        lg={
            'epoch': epo+1,
            'tr_accuracy': tr_acc,
            'val_accuracy': val_acc,
            'tr_loss': tr_ls,
            'val_loss': val_ls
        }
        wandb.log(lg)

In [11]:
sweep_config = {
  "method": "grid",  # Use grid search for this example
  "name": "Q1 Sweep",
  "metric": {"goal": "maximize", "name": "val_accuracy"},
  "parameters": {
    "data_augmentation":{"values": [True]},  # List of boolean values
    "batch_normalization":{"values": [False]},  # List of boolean values
    "filters":{"values": [64]},  # List of filter values for first layer
    "filter_org":{"values": ["half"]},  # List of filter organization options.. , "half", "double"
    "dropout":{"values": [0.2]},  # Dropout rates
    "activation":{"values": ["relu"]},  # Activation functions... , "tanh", "leaky_relu"
    "batch_size":{"values": [64]},
    "learning_rate":{"values": [0.001]},
    "epochs":{"values": [10]},
    "dim":{"values": [256]},
    "conv_kernel_size":{"values": [3]},
    "train_data_dir":{"values": ["./data/train"]},
    "test_data_dir":{"values": ["./data/val"]}
  }
}


In [12]:
# wandb.init()
sweep_id = wandb.sweep(sweep_config, project="cs6910_assignment2")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 468ahafa
Sweep URL: https://wandb.ai/cs23m070/cs6910_assignment2/sweeps/468ahafa


In [13]:
wandb.agent(sweep_id, function=train, count=1)
wandb.finish()

wandb: Agent Starting Run: iooz0ffa with config:
wandb: 	activation: relu
wandb: 	batch_normalization: False
wandb: 	batch_size: 64
wandb: 	conv_kernel_size: 3
wandb: 	data_augmentation: True
wandb: 	dim: 256
wandb: 	dropout: 0.2
wandb: 	epochs: 10
wandb: 	filter_org: half
wandb: 	filters: 64
wandb: 	learning_rate: 0.001
wandb: 	test_data_dir: ./data/val
wandb: 	train_data_dir: ./data/train
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: cs23m070. Use `wandb login --relogin` to force relogin


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
Train Loss --> 2.284712553024292
Train Accuracy --> 0.123625
Validation Loss --> 2.2659122943878174
Validation Accuracy --> 0.17508754377188596
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
Train Loss --> 2.2340598106384277
Train Accuracy --> 0.16775
Validation 

epoch,▁▂▃▃▄▅▆▆▇█
tr_accuracy,▁▃▄▅▆▇▇▇██
tr_loss,█▇▆▅▄▃▃▂▂▁
val_accuracy,▁▂▄▃▆▅▆▇▇█
val_loss,█▇▆▇▄▄▃▂▂▁
epoch,10
tr_accuracy,0.2715
tr_loss,2.02832
val_accuracy,0.29265
val_loss,2.01645


In [1]:
# import torch
# if torch.backends.mps.is_available():
#     mps_device = torch.device("mps")
#     x = torch.ones(1, device=mps_device)
#     print (x)
# else:
#     print ("MPS device not found.")

tensor([1.], device='mps:0')


In [8]:
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# print(device)

mps


In [103]:
# wandb.finish()